In [1]:
from psycopg2 import connect as pgconnect
import snowflake.connector
import pandas as pd
import string

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (20,20)

from IPython.display import SVG
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [9]:
recipe_all = pd.read_csv('/Users/xun/Documents/Thesis/recipe_data/recipe_data_v2.csv')
recipe_all.shape # (2060, 15)

(1522, 10)

In [4]:
recipe_all.head()

,publishdate,content_id,title_en,cuisine,difficulty,mealtype,diettype,cooking_method,steps,ingredients
0,2017-04-29,R702,Braised chicken with couscous,african,medium,main,non-vegetarian,"marinating,roasting,slow-cooking,steaming","""Deglaze with some of the chicken stock."" ""In ...","{""chicken legs"",""chicken stock (divided)"",""cou..."
1,2017-11-05,R943,Lamb and quince tagine,african,medium,main,non-vegetarian,NaN,"""Heat olive oil and butter in a tagine or larg...","{""butter"",""cilantro for garnishing"",""ground ci..."
2,2018-07-24,R1295,East African-style coconut chicken,african,medium,main,non-vegetarian,sauteing,"""Add the onions, garlic, and ginger to the pot...","{""canned crushed tomatoes"",""chicken legs"",""cin..."
3,2017-02-02,R574,Moroccan-inspired chickpea stew,african,easy,main,vegetarian,stewing,"""Add chickpeas to stew and let simmer for appr...","{""apple cider vinegar (optional)"",""black peppe..."
4,2015-06-29,R256,Coleslaw with cranberries,american,easy,others,vegetarian,marinating,"""Finely chop parsley. Roughly chop cranberries...","{""buttermilk"",""cabbage"",""carrots"",""cranberries..."


In [11]:
list(recipe_all.cuisine.unique())

['african',
 'american',
 'chinese',
 'french',
 'german',
 'greek',
 'indian',
 'italian',
 'japanese',
 'korean',
 'middle-eastern',
 'others',
 'spanish-and-portuguese',
 'thai',
 'vietnamese']

In [12]:
recipe_all['ingredients'][0]
recipe_all['steps'][0]

'{"chicken legs","chicken stock (divided)","couscous","cumin","garlic","green bell pepper","honey","lemon juice","onions","parsley for garnish","pepper","raisins","rapeseed oil (divided)","red bell pepper","salt","spicy paprika","turmeric","yellow bell pepper"}'

'"Deglaze with some of the chicken stock." "In the meantime, wash bell peppers. Remove seeds and cut into strips. Rinse raisins under cold water. Add cut bell peppers, raisins, couscous, and remaining chicken stock to tagine, cover, and continue to simmer for approx. 10 min." "Peel and slice onions and garlic. Remove chicken legs from marinade and carefully drain. Heat oil in the bottom of the tagine over medium-high heat. Add chicken legs, garlic, and onion and fry until deep golden-brown on all sides." "Place the lid on the tagine. Let simmer for approx. 80 min." "Season with lemon juice, salt, and pepper. Garnish with parsley. Serve directly in the tagine. Enjoy!" "Wash and dry chicken legs. Mix a part of rapeseed oil with honey until smooth. Add turmeric, cumin, cinnamon, spicy paprika, salt, and pepper. In a large bowl or a freezer bag, toss chicken legs in marinade. Seal or cover and marinate overnight in the refrigerator."'

In [13]:
recipe_all['ingredients']=recipe_all['ingredients'].str.replace('{','')
recipe_all['ingredients']=recipe_all['ingredients'].str.replace('}','')
recipe_all['ingredients']=recipe_all['ingredients'].str.replace('"','')
recipe_all['ingredients']= recipe_all['ingredients'].str.split(',')
recipe_all['steps']=recipe_all['steps'].str.replace('"','')

In [14]:
def count_word_from_text(text):
    text_to_list = text.split()
    count_word = len(text_to_list)
    return count_word

In [15]:
recipe_all['steps'][0]
count_word_from_text(recipe_all['steps'][0])

'Deglaze with some of the chicken stock. In the meantime, wash bell peppers. Remove seeds and cut into strips. Rinse raisins under cold water. Add cut bell peppers, raisins, couscous, and remaining chicken stock to tagine, cover, and continue to simmer for approx. 10 min. Peel and slice onions and garlic. Remove chicken legs from marinade and carefully drain. Heat oil in the bottom of the tagine over medium-high heat. Add chicken legs, garlic, and onion and fry until deep golden-brown on all sides. Place the lid on the tagine. Let simmer for approx. 80 min. Season with lemon juice, salt, and pepper. Garnish with parsley. Serve directly in the tagine. Enjoy! Wash and dry chicken legs. Mix a part of rapeseed oil with honey until smooth. Add turmeric, cumin, cinnamon, spicy paprika, salt, and pepper. In a large bowl or a freezer bag, toss chicken legs in marinade. Seal or cover and marinate overnight in the refrigerator.'

158

In [16]:
recipe_all['ingredients'][0]
len(recipe_all.ingredients[0])

['chicken legs',
 'chicken stock (divided)',
 'couscous',
 'cumin',
 'garlic',
 'green bell pepper',
 'honey',
 'lemon juice',
 'onions',
 'parsley for garnish',
 'pepper',
 'raisins',
 'rapeseed oil (divided)',
 'red bell pepper',
 'salt',
 'spicy paprika',
 'turmeric',
 'yellow bell pepper']

18

In [17]:
#get the mean of len(steps) and #ingredients
recipe_all['#ingredients'] = recipe_all.ingredients.apply(len)
recipe_all['steps_length'] = recipe_all.steps.apply(count_word_from_text)

In [18]:
recipe_all.iloc[0:1,-4:]

,steps,ingredients,#ingredients,steps_length
0,Deglaze with some of the chicken stock. In the...,"[chicken legs, chicken stock (divided), cousco...",18,158


In [19]:
recipe_all.head().applymap(type)

,publishdate,content_id,title_en,cuisine,difficulty,mealtype,diettype,cooking_method,steps,ingredients,#ingredients,steps_length
0,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'int'>,<class 'int'>
1,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'list'>,<class 'int'>,<class 'int'>
2,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'int'>,<class 'int'>
3,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'int'>,<class 'int'>
4,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'int'>,<class 'int'>


In [20]:
recipe_all['steps_length'].mean()

150.2437582128778

In [21]:
recipe_all['#ingredients'].mean()

12.170827858081472

In [ ]:
##################
#average length of recipe step text is 817 characters, 150 words 
#and average number of ingredient is 11
################

In [24]:
recipe_all.to_csv(r'/Users/xun/Documents/Thesis/recipe_data/recipe_data_processed.csv', index = None)
